In [1]:
import pandas as pd 
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Imputer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from scipy.stats import pearsonr
from sklearn.tree import DecisionTreeClassifier
import itertools
from copy import copy
import lightgbm as lgb
import seaborn as sns
from sklearn.model_selection import KFold
from itertools import combinations
import numpy as np
from matplotlib import pyplot as plt

/Users/GoormanMcBook/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [1]:
import glob

In [5]:
glob.glob("ml5/*")

['ml5/test.csv', 'ml5/train.csv']

In [2]:
train = pd.read_csv("ml5/train.csv", sep=";")
test = pd.read_csv("ml5/test.csv", sep=";")
test["smoke"] = test["smoke"].apply(lambda x: np.nan if (x == "None")  else int(x))
test["alco"] = test["alco"].apply(lambda x: np.nan if (x == "None")  else int(x))
test["active"] = test["active"].apply(lambda x: np.nan if (x == "None")  else int(x))

In [3]:
train.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [4]:
train.active.value_counts()

1    56261
0    13739
Name: active, dtype: int64

In [5]:
categorical_f = ["cholesterol", "gluc"]
bin_f = ["gender", "alco", "smoke", "active"]
numeric_f = ["age", "height", "weight", "ap_hi", "ap_lo"]

In [6]:
def get_int_columns(columns_list):
    int_columns = []
    for f in columns_list:
        if all(data_train[f].dropna().astype("int64").astype(float) == data_train[f].dropna())\
           and (len(data_train[f].value_counts()) > 1):
            int_columns.append(f)
    return int_columns

In [61]:
def null_outliers(df, c_list):
    df = df.copy()
    for c in c_list:
        if (len(df[c].value_counts()) != 2):
            mean = df[c].mean()
            std = df[c].std()
            df.loc[np.abs(df[c]-mean)>(3.2*std), c] = np.nan
    return df


def process_table(df):
    df = df.copy()
    df = null_outliers(df, df.columns)
    return df

In [63]:
data_train.smoke.value_counts(dropna = False)

0    63831
1     6169
Name: smoke, dtype: int64

In [62]:
data_train = process_table(train)
data_test = process_table(test)

In [64]:
int_columns = get_int_columns(data_train.columns)

In [65]:
def get_proba(d):
    d = d.astype(float)
    sum_v = sum(d.values)
    for i in d.keys():
        d[i] *= (1.0 / sum_v)
    return d

def find_best_val(val, d):
    keys = d.keys()
    best_val = keys[0]
    min_dist = abs(d[keys[0]] - val)
    for k in keys[1:]:
        dist = abs(d[k] - val)
        if dist < min_dist:
            min_dist = dist
            best_val = k
    return best_val

def get_bad_keys_dict(bad_keys, test_proba_dict, train_proba_dict):
    bad_keys_dict = {}
    for k in bad_keys:
        bad_keys_dict[k] = find_best_val(test_proba_dict[k], train_proba_dict)
    return bad_keys_dict

In [66]:
def get_categorical_features(train, test, c_columns):
    train = train.copy()
    test = test.copy()
    for f in tqdm_notebook(c_columns):
        train_d = get_proba(train[f].value_counts())
        train.loc[train[f].isnull(), f] = np.random.choice(train_d.keys(), size = len(train[train[f].isnull()]), 
                                                           p=train_d.values)
        
        test.loc[test[f].isnull(), f] = np.random.choice(train_d.keys(), size = len(test[test[f].isnull()]), 
                                                           p=train_d.values)
        test_d = get_proba(test[f].value_counts())
        bad_keys = list(set(test_d.keys()).difference(set(train_d.keys())))
        bad_keys_dict = get_bad_keys_dict(bad_keys, test_d, train_d)
        test.loc[test[f].isin(bad_keys), f] = test.loc[test[f].isin(bad_keys), f].apply(lambda x: bad_keys_dict[x])
        
        le = LabelEncoder()
        train[f] = le.fit_transform(train[f])
        test[f] = le.transform(test[f])
        
        enc = OneHotEncoder(sparse=False)
        o_f_train = enc.fit_transform(train[[f]])
        o_f_test = enc.transform(test[[f]])
        column_names = [f + "_"+str(i) for i in range(o_f_train.shape[1])]
        df_t_train = pd.DataFrame(data=o_f_train, columns=column_names)
        df_t_test = pd.DataFrame(data=o_f_test, columns=column_names)
        train = train.join(df_t_train)
        test = test.join(df_t_test)
    return train, test

In [67]:
def get_numeric_features(train, test, n_list, log_f):
    train = train.copy()
    test = test.copy()
    for f in n_list:
        if (f in int_columns):
            v_c = train[f].value_counts()
            train_d = get_proba(v_c)
            
            train.loc[train[f].isnull(), f] = np.random.choice(train_d.keys(), size = len(train[train[f].isnull()]), 
                                                           p=train_d.values)
            test.loc[test[f].isnull(), f] = np.random.choice(train_d.keys(), size = len(test[test[f].isnull()]), 
                                                           p=train_d.values)
        else:
            mean = train[f].astype(float).mean()
            std  = train[f].astype(float).std()
            min_f = train[f].min()
            max_f = train[f].max()
            train.loc[train[f].isnull(), f] = pd.Series(np.random.normal(mean, std, len(train[train[f].isnull()]))).clip(min_f ,max_f).values
            test.loc[test[f].isnull(), f] = pd.Series(np.random.normal(mean, std, len(test[test[f].isnull()]))).clip(min_f, max_f).values
        
        if f in log_f:
            max_train = -train[f].min() + 1
            max_test = -test[f].min() + 1
            train["log_"+f] = train[f].apply(lambda x: np.log(x+max_train))
            test["log_"+f] = test[f].apply(lambda x: np.log(x+max_test))
    
    return train, test

In [68]:
def get_numeric_mean_nan(train, test, n_list, log_f):
    train = train.copy()
    test = test.copy()
    for f in n_list:
        mean = train[f].astype(float).mean()
        train.loc[train[f].isnull(), f] = mean
        test.loc[test[f].isnull(), f] = mean
        if f in log_f:
            max_train = -train[f].min() + 1
            max_test = -test[f].min() + 1
            train["log_"+f] = train[f].apply(lambda x: np.log(x+max_train))
            test["log_"+f] = test[f].apply(lambda x: np.log(x+max_test))
    
    return train, test

In [69]:
def get_sp_bool_f(df, f):
    df = df.copy()
    for i, j in combinations(f, 2):
        if i[:5] != j[:5]:
            df[i+"&" +j] = df[i].astype(int) & df[j].astype(int)
            df[i+"|" +j] = df[i].astype(int) | df[j].astype(int)
            df[i+"^" +j] = df[i].astype(int) ^ df[j].astype(int)
    return df

def get_sp_num_f(df, f_list):
    df = df.copy()
    for f in f_list:
        df["sqrt_"+f] = np.sqrt(abs(df[f]))
        df["sqr_"+f] = df[f] * df[f]
#    print len(f_list)
    for f1, f2 in tqdm_notebook(combinations(f_list, 2)):
        if (f1 != ("norm_" + f2)) and (f2 != ("norm_" + f1)):
            df[f1+"_sub_"+f2] = df[f1] - df[f2]
            df[f1+"_abs_dif_"+f2] = df[f1] - df[f2]
            df[f1+"_sum_"+f2] = df[f1] + df[f2] 
            df[f1+"_mul_"+f2] = df[f1] * df[f2]
            if len(df[df[f2]==0]) * 1.0 / len(df) < 0.2:
                df[f1+"_div_"+f2] = df[f1].astype(float) / df[f2]
                df[f1+"_div_"+f2] = df[f1+"_div_"+f2].replace(np.inf, np.nan)
                df[f1+"_div_"+f2] = df[f1+"_div_"+f2].fillna(0)
                
            if len(df[df[f1]==0]) * 1.0 / len(df) < 0.2:
                df[f2+"_div_"+f1] = df[f2].astype(float) / df[f1]
                df[f2+"_div_"+f1] = df[f2+"_div_"+f1].replace(np.inf, np.nan)
                df[f2+"_div_"+f1] = df[f2+"_div_"+f1].fillna(0)
                
#    print "sp_f done"
    return df

In [70]:
def check_c_column(column, c_column_names):
    for c in c_column_names:
        ind = column.find(c+"_")
        if (ind == 0) and column[ind+len(c)+1:].isdigit():
            return True
    return False

In [71]:
def create_features(train, test, c_columns, n_columns, sp_bin_f=[],
                    sp_c_f=None, fill_mean=False,
                    sp_n_f=None, log_f=[]):
    
    train = train.copy()
    test = test.copy()
    full_f = c_columns + n_columns
    
    train = train[full_f + ["id", "cardio"]]
    
    if "target" in test.columns:
        test = test[full_f + ["id", "cardio"]]
    else:
        test = test[full_f + ["id"]]
        
    train, test = get_categorical_features(train, test, c_columns)
#    print "c done"
    if not fill_mean:
        train, test = get_numeric_features(train, test, n_columns, log_f)
    else:
        train, test = get_numeric_mean_nan(train, test, n_columns, log_f)
#    print "n done"
    
    sp_bin_f = list(copy(sp_bin_f))
    
    if sp_c_f is not None:
        
        sp_bin_f += [i for i in train.columns if check_c_column(i, sp_c_f)]
        
    if len(sp_bin_f) != 0:
        sp_bin_f = list(set(sp_bin_f))
        
    if sp_bin_f is not None:
        train = get_sp_bool_f(train, sp_bin_f)
        test = get_sp_bool_f(test, sp_bin_f)
#    print "bin done"
    
      
    if sp_n_f is not None:
        train = get_sp_num_f(train, sp_n_f)
#        print "ok here"
        test = get_sp_num_f(test, sp_n_f)
#    print "sp done"
    
    return train, test

In [72]:
sel_c_columns = categorical_f + bin_f
sel_n_columns = numeric_f
#sel_sp_bin_f = bin_f
sel_sp_bin_f = ["gender", "alco", "active"]
sel_sp_c_f = categorical_f
log_f = numeric_f
sel_sp_n_f = numeric_f

In [73]:
sp_params = {"sp_bin_f": sel_sp_bin_f, "sp_c_f": sel_sp_c_f, "sp_n_f":sel_sp_n_f, "log_f": log_f}

In [74]:
def cv_tt(df, target, cv, c_f_list, n_list, sp_params={}):
    tt = []
    df = df.copy()
    y = df[target]
    data = df.copy()
    
    for itrain, itest in tqdm_notebook(cv):
        train_data = data.loc[itrain,:].copy()
        test_data = data.loc[itest,:].copy()
        train_data.reset_index(inplace=True)
        test_data.reset_index(inplace=True)
        train_data, test_data = create_features(train_data, test_data, c_f_list, n_list, **sp_params)
        fold = {"xtrain" : train_data, "ytrain" : y[itrain], 
                "xtest" : test_data, "ytest" : y[itest]}
        tt.append(fold)

    return tt

In [75]:
train.columns

Index([u'id', u'age', u'gender', u'height', u'weight', u'ap_hi', u'ap_lo',
       u'cholesterol', u'gluc', u'smoke', u'alco', u'active', u'cardio'],
      dtype='object')

In [76]:
kfold = KFold(n_splits=10)
cv = list(kfold.split(data_train))
tt = cv_tt(data_train, "smoke", cv, categorical_f, ["gender", "alco", "active"] + numeric_f, sp_params=sp_params)

In [77]:
data_train.head()

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0.0,18393.0,2,168.0,62.0,110.0,80.0,1.0,1.0,0,0,1,0
1,1.0,20228.0,1,156.0,85.0,140.0,90.0,3.0,1.0,0,0,1,1
2,2.0,18857.0,1,165.0,64.0,130.0,70.0,3.0,1.0,0,0,0,1
3,3.0,17623.0,2,169.0,82.0,150.0,100.0,1.0,1.0,0,0,1,1
4,4.0,17474.0,1,156.0,56.0,100.0,60.0,1.0,1.0,0,0,0,0


In [78]:
def fit_predict(clf, xtrain, ytrain, xtest, ytest, clf_params, 
                fit_params, do_scaling, xgbfit, f_l):
    if do_scaling:
        scaler = StandardScaler()
        xtrain = scaler.fit_transform(xtrain)
        xtest = scaler.transform(xtest)
        
    if xgbfit:
        if ytest is not None:
            fit_params["eval_set"] = \
                [(xtrain, ytrain), (xtest, ytest)]
        else:
            fit_params["eval_set"] = [(xtrain, ytrain)]
   
    c = clf(**clf_params)
    c.fit(xtrain, ytrain, **fit_params)
    
    p_tst = c.predict(xtest)
    pr_tst = c.predict_proba(xtest)[:,1]
    
    if do_scaling:
        a = c.coef_[0][c.coef_[0]!=0]
        b = np.array(f_l)[c.coef_[0]!=0]
        f_weights = sorted(zip(a, b), key = lambda x : abs(x[0]), reverse=True)
    else:
        
        a = c.feature_importances_[c.feature_importances_ !=0]
        b = np.array(f_l)[c.feature_importances_ !=0]
        f_weights = sorted(zip(a, b), key = lambda x : abs(x[0]), reverse=True)
    
    return f_weights, p_tst, pr_tst

In [79]:
def get_f(all_f, f_names):
    f_l = []
    for f in f_names:
        for i in all_f:
            if (i == f) and (f+"_0" not in all_f):
                f_l.append(i)
                break
            else:
                ind = i.find(f+"_")
                if (ind == 0) and i[ind+len(f)+1:].isdigit():
                    f_l.append(i)
    return f_l

In [80]:
def get_prediction(train, test, f_names, clf_type, clf_params, fit_params, do_scaling, xgbfit, 
                   nnfit=False, lgbm=False, real_test=False):
    y_train = train["cardio"]
    train_data = train.copy()
    
    if not real_test:
        y_test = test["cardio"]
        test_data = test.copy()
        
    else:
        y_test = None
        test_data = test.copy()
    
    all_f = list(train.columns)
        
    if lgbm:
        f_l = f_names
    else:
        f_l = get_f(all_f, f_names)
    print "prediction"
    print f_l
    f_weights, p_tst, pr_tst = fit_predict(clf_type, train_data[f_l], y_train, 
                test_data[f_l], y_test, clf_params, 
                fit_params, do_scaling, xgbfit, f_l)
    if not real_test:
        log_loss_score = log_loss(y_test, pr_tst)
    else:
        log_loss_score = None
        
    res = { "p_test" : p_tst, "pr_test" : pr_tst, 
           "log_loss": log_loss_score,
           "f_weights": f_weights}
    return res

In [81]:
def cv_fit_predict(clf, cv_tt, f_names, clf_params = {}, fit_params = {}, 
                   do_scaling = False, xgbfit = False, nnfit = False, lgbm=False):
    cv_answer = []
    
    for fold in tqdm_notebook(cv_tt):
        all_f = list(fold["xtrain"].columns)
        
        if lgbm:
            f_l = f_names
        else:
            f_l = get_f(all_f, f_names)
        
        f_weights, p_tst, pr_tst = fit_predict(clf, fold["xtrain"][f_l], fold["ytrain"], 
                                      fold["xtest"][f_l], fold["ytest"], 
                                      clf_params, fit_params, do_scaling, xgbfit, f_l)
        log_loss_score = log_loss(fold["ytest"], pr_tst)
        print log_loss_score
        tt_answer = {"p_test" : p_tst, "pr_test" : pr_tst, 
                     "log_loss": log_loss_score,
                     "f_weights": f_weights}
        cv_answer.append(tt_answer)
    
    return cv_answer

In [82]:
div_columns = [i for i in data_test_f.columns if "_div_" in i]
mul_columns = [i for i in data_train_f.columns if "_mul_" in i]

In [85]:
lrclf_params = {"C": 0.5, "n_jobs" : -1, "penalty": "l2"}
lrfit_params = {}
#lr_f = [u'cholesterol', u'gluc', u'gender', u'alco', u'smoke', u'active',
#       u'age', u'height', u'weight', u'ap_hi', u'ap_lo'] + div_columns + mul_columns
lr_f = [u'cholesterol', u'gluc', u'gender', u'alco', u'active',
       u'age', u'height', u'weight', u'ap_hi', u'ap_lo'] + div_columns + mul_columns
cv_lr_1 = cv_fit_predict(LogisticRegression, tt, lr_f, lrclf_params, lrfit_params, 
                          do_scaling = True, xgbfit = False, lgbm=False)
print "mean_score", np.mean([i["log_loss"] for i in cv_lr_1])

0.2270463485
0.231441750939
0.223612116644
0.220147634879
0.202565274737
0.21239664393
0.212650534971
0.218288554411
0.221745133462
0.219637287085

mean_score 0.218953127956


In [29]:
print np.mean([i["log_loss"] for i in cv_lr_1])

0.557592366003


In [86]:
xgbclf_params = {"objective":"binary:logistic", "max_depth":7, "learning_rate":0.01,
                 "n_estimators": 500, "nthread":8, "base_score":0.5, "seed":0, "missing":None}

xgbfit_params = {"eval_metric":"logloss", "early_stopping_rounds":90000, "verbose":True}
xgb_f = [u'cholesterol', u'gluc', u'gender', u'alco', u'active',
       u'age', u'height', u'weight', u'ap_hi', u'ap_lo'] + div_columns + mul_columns

In [87]:
cv_xgb_1 = cv_fit_predict(XGBClassifier, tt, xgb_f, xgbclf_params, xgbfit_params, 
                          do_scaling = False, xgbfit = True, lgbm=False)

[0]	validation_0-logloss:0.685628	validation_1-logloss:0.685738
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 90000 rounds.
[1]	validation_0-logloss:0.678258	validation_1-logloss:0.678474
[2]	validation_0-logloss:0.671033	validation_1-logloss:0.671355
[3]	validation_0-logloss:0.663949	validation_1-logloss:0.664377
[4]	validation_0-logloss:0.657	validation_1-logloss:0.65753
[5]	validation_0-logloss:0.650185	validation_1-logloss:0.650818
[6]	validation_0-logloss:0.643499	validation_1-logloss:0.644229
[7]	validation_0-logloss:0.636939	validation_1-logloss:0.637773
[8]	validation_0-logloss:0.6305	validation_1-logloss:0.63144
[9]	validation_0-logloss:0.624184	validation_1-logloss:0.62522
[10]	validation_0-logloss:0.617982	validation_1-logloss:0.619116
[11]	validation_0-logloss:0.611895	validation_1-logloss:0.613119
[12]	validation_0-logloss:0.605915	validation_1-logloss:0.607245
[13]	

[124]	validation_0-logloss:0.293567	validation_1-logloss:0.303662
[125]	validation_0-logloss:0.292418	validation_1-logloss:0.30258
[126]	validation_0-logloss:0.291285	validation_1-logloss:0.301513
[127]	validation_0-logloss:0.290155	validation_1-logloss:0.300474
[128]	validation_0-logloss:0.289039	validation_1-logloss:0.299442
[129]	validation_0-logloss:0.287944	validation_1-logloss:0.298413
[130]	validation_0-logloss:0.286857	validation_1-logloss:0.297404
[131]	validation_0-logloss:0.285787	validation_1-logloss:0.29641
[132]	validation_0-logloss:0.284734	validation_1-logloss:0.295418
[133]	validation_0-logloss:0.283688	validation_1-logloss:0.294457
[134]	validation_0-logloss:0.282656	validation_1-logloss:0.293498
[135]	validation_0-logloss:0.281638	validation_1-logloss:0.292551
[136]	validation_0-logloss:0.280635	validation_1-logloss:0.29162
[137]	validation_0-logloss:0.279643	validation_1-logloss:0.290702
[138]	validation_0-logloss:0.278663	validation_1-logloss:0.289795
[139]	validat

[249]	validation_0-logloss:0.219301	validation_1-logloss:0.237869
[250]	validation_0-logloss:0.219043	validation_1-logloss:0.237672
[251]	validation_0-logloss:0.21879	validation_1-logloss:0.237468
[252]	validation_0-logloss:0.218543	validation_1-logloss:0.237275
[253]	validation_0-logloss:0.218297	validation_1-logloss:0.237091
[254]	validation_0-logloss:0.218049	validation_1-logloss:0.236906
[255]	validation_0-logloss:0.217804	validation_1-logloss:0.23672
[256]	validation_0-logloss:0.217562	validation_1-logloss:0.236523
[257]	validation_0-logloss:0.217328	validation_1-logloss:0.236347
[258]	validation_0-logloss:0.217098	validation_1-logloss:0.236165
[259]	validation_0-logloss:0.216863	validation_1-logloss:0.235994
[260]	validation_0-logloss:0.216643	validation_1-logloss:0.235824
[261]	validation_0-logloss:0.216418	validation_1-logloss:0.235658
[262]	validation_0-logloss:0.216201	validation_1-logloss:0.235498
[263]	validation_0-logloss:0.215978	validation_1-logloss:0.235336
[264]	valida

[374]	validation_0-logloss:0.201444	validation_1-logloss:0.22673
[375]	validation_0-logloss:0.201382	validation_1-logloss:0.226709
[376]	validation_0-logloss:0.201297	validation_1-logloss:0.226679
[377]	validation_0-logloss:0.201219	validation_1-logloss:0.22664
[378]	validation_0-logloss:0.201157	validation_1-logloss:0.22662
[379]	validation_0-logloss:0.201092	validation_1-logloss:0.226595
[380]	validation_0-logloss:0.200995	validation_1-logloss:0.226573
[381]	validation_0-logloss:0.20092	validation_1-logloss:0.226544
[382]	validation_0-logloss:0.200841	validation_1-logloss:0.226526
[383]	validation_0-logloss:0.20078	validation_1-logloss:0.226497
[384]	validation_0-logloss:0.200704	validation_1-logloss:0.226475
[385]	validation_0-logloss:0.200645	validation_1-logloss:0.226453
[386]	validation_0-logloss:0.200568	validation_1-logloss:0.226438
[387]	validation_0-logloss:0.200496	validation_1-logloss:0.226428
[388]	validation_0-logloss:0.200431	validation_1-logloss:0.226414
[389]	validatio

[499]	validation_0-logloss:0.195649	validation_1-logloss:0.2256
0.225599537594
[0]	validation_0-logloss:0.685623	validation_1-logloss:0.685727
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 90000 rounds.
[1]	validation_0-logloss:0.678248	validation_1-logloss:0.678456
[2]	validation_0-logloss:0.671018	validation_1-logloss:0.671329
[3]	validation_0-logloss:0.663929	validation_1-logloss:0.664345
[4]	validation_0-logloss:0.656978	validation_1-logloss:0.65749
[5]	validation_0-logloss:0.65016	validation_1-logloss:0.650778
[6]	validation_0-logloss:0.643472	validation_1-logloss:0.644186
[7]	validation_0-logloss:0.63691	validation_1-logloss:0.637727
[8]	validation_0-logloss:0.630471	validation_1-logloss:0.631386
[9]	validation_0-logloss:0.62415	validation_1-logloss:0.625156
[10]	validation_0-logloss:0.617948	validation_1-logloss:0.619055
[11]	validation_0-logloss:0.611858	validation_1-logl

...

[491]	validation_0-logloss:0.195948	validation_1-logloss:0.221725
[492]	validation_0-logloss:0.195908	validation_1-logloss:0.221719
[493]	validation_0-logloss:0.19588	validation_1-logloss:0.221715
[494]	validation_0-logloss:0.195842	validation_1-logloss:0.221713
[495]	validation_0-logloss:0.19582	validation_1-logloss:0.221712
[496]	validation_0-logloss:0.195793	validation_1-logloss:0.221704
[497]	validation_0-logloss:0.195766	validation_1-logloss:0.221704
[498]	validation_0-logloss:0.195714	validation_1-logloss:0.221698
[499]	validation_0-logloss:0.19568	validation_1-logloss:0.221702
0.221702326493
[0]	validation_0-logloss:0.685643	validation_1-logloss:0.685714
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 90000 rounds.
[1]	validation_0-logloss:0.678287	validation_1-logloss:0.678441
[2]	validation_0-logloss:0.671077	validation_1-logloss:0.671303
[3]	validation_0-logloss:0.664005	

[115]	validation_0-logloss:0.305197	validation_1-logloss:0.312772
[116]	validation_0-logloss:0.303899	validation_1-logloss:0.311533
[117]	validation_0-logloss:0.302624	validation_1-logloss:0.310314
[118]	validation_0-logloss:0.301364	validation_1-logloss:0.309117
[119]	validation_0-logloss:0.300119	validation_1-logloss:0.30794
[120]	validation_0-logloss:0.29889	validation_1-logloss:0.306772
[121]	validation_0-logloss:0.297678	validation_1-logloss:0.30563
[122]	validation_0-logloss:0.296472	validation_1-logloss:0.304499
[123]	validation_0-logloss:0.295295	validation_1-logloss:0.303369
[124]	validation_0-logloss:0.294133	validation_1-logloss:0.302267
[125]	validation_0-logloss:0.292978	validation_1-logloss:0.301176
[126]	validation_0-logloss:0.29184	validation_1-logloss:0.300103
[127]	validation_0-logloss:0.290722	validation_1-logloss:0.299039
[128]	validation_0-logloss:0.289617	validation_1-logloss:0.297994
[129]	validation_0-logloss:0.288523	validation_1-logloss:0.296967
[130]	validati

[240]	validation_0-logloss:0.222189	validation_1-logloss:0.236906
[241]	validation_0-logloss:0.221907	validation_1-logloss:0.236666
[242]	validation_0-logloss:0.221627	validation_1-logloss:0.236436
[243]	validation_0-logloss:0.221344	validation_1-logloss:0.236203
[244]	validation_0-logloss:0.221068	validation_1-logloss:0.235954
[245]	validation_0-logloss:0.220785	validation_1-logloss:0.235724
[246]	validation_0-logloss:0.220519	validation_1-logloss:0.235494
[247]	validation_0-logloss:0.220251	validation_1-logloss:0.235281
[248]	validation_0-logloss:0.219984	validation_1-logloss:0.235054
[249]	validation_0-logloss:0.219725	validation_1-logloss:0.234839
[250]	validation_0-logloss:0.219475	validation_1-logloss:0.234636
[251]	validation_0-logloss:0.219227	validation_1-logloss:0.234431
[252]	validation_0-logloss:0.218981	validation_1-logloss:0.234234
[253]	validation_0-logloss:0.218727	validation_1-logloss:0.234028
[254]	validation_0-logloss:0.218486	validation_1-logloss:0.233834
[255]	vali

[365]	validation_0-logloss:0.202883	validation_1-logloss:0.222805
[366]	validation_0-logloss:0.202805	validation_1-logloss:0.222765
[367]	validation_0-logloss:0.202728	validation_1-logloss:0.222726
[368]	validation_0-logloss:0.202648	validation_1-logloss:0.222682
[369]	validation_0-logloss:0.202566	validation_1-logloss:0.222635
[370]	validation_0-logloss:0.202497	validation_1-logloss:0.222604
[371]	validation_0-logloss:0.202421	validation_1-logloss:0.222563
[372]	validation_0-logloss:0.202351	validation_1-logloss:0.222538
[373]	validation_0-logloss:0.202275	validation_1-logloss:0.222503
[374]	validation_0-logloss:0.2022	validation_1-logloss:0.222473
[375]	validation_0-logloss:0.202123	validation_1-logloss:0.222434
[376]	validation_0-logloss:0.202059	validation_1-logloss:0.222401
[377]	validation_0-logloss:0.201987	validation_1-logloss:0.222358
[378]	validation_0-logloss:0.201914	validation_1-logloss:0.222333
[379]	validation_0-logloss:0.201839	validation_1-logloss:0.222302
[380]	valida

[490]	validation_0-logloss:0.1967	validation_1-logloss:0.221011
[491]	validation_0-logloss:0.196679	validation_1-logloss:0.221012
[492]	validation_0-logloss:0.196655	validation_1-logloss:0.221002
[493]	validation_0-logloss:0.196636	validation_1-logloss:0.220996
[494]	validation_0-logloss:0.196612	validation_1-logloss:0.22099
[495]	validation_0-logloss:0.196569	validation_1-logloss:0.22099
[496]	validation_0-logloss:0.196546	validation_1-logloss:0.22098
[497]	validation_0-logloss:0.196526	validation_1-logloss:0.220981
[498]	validation_0-logloss:0.196499	validation_1-logloss:0.220979
[499]	validation_0-logloss:0.19647	validation_1-logloss:0.220986
0.220986079051



In [ ]:
print "mean_score", np.mean([i["log_loss"] for i in cv_xgb_1])

In [ ]:
res_test_xgb = get_prediction(data_train_f, data_test_f, xgb_f, XGBClassifier, xgbclf_params, xgbfit_params, 
                              do_scaling = False, xgbfit = True, lgbm=False)